In [ ]:
import sys, os; sys.path.append('..')
import pyzx as zx
import random
from fractions import Fraction
%config InlineBackend.figure_format = 'svg'

In [ ]:
c = zx.qasm("""
qreg q[3];

ccz q[0],q[1],q[2];
h q[2];
t q[2];
ccz q[0],q[1],q[2];
h q[2];
t q[1];
ccz q[0],q[1],q[2];
s q[2];
ccx q[0],q[1],q[2];
""")
zx.draw(c, labels=True)

In [ ]:
h = c.to_graph(zh=True)
zx.simplify.spider_simp(h)
zx.hsimplify.to_hbox(h)
m = zx.hrules.match_hpivot(h)
print(m)
display(zx.draw(h,labels=True))
zx.hrules.hpivot(h,m)
display(zx.draw(h,labels=True))

In [ ]:
zx.hsimplify.hpivot_simp(h)
zx.d3.draw(h, labels=True)

In [ ]:
random.seed(1338)

qs = 5
d = 10
p_t = 0.2
p_h = 0.3

for j in range(50):
    c = zx.Circuit(qs)
    for i in range(d):
        p = random.random()
        if p < p_h:
            c.add_gate(zx.circuit.HAD(random.randint(0,qs-1)))
        elif p < p_t + p_h:
            c.add_gate(zx.circuit.T(random.randint(0,qs-1)))
        else:
            q = sorted(random.sample(range(qs), 3))
            c.add_gate(zx.circuit.CCZ(q[0],q[1],q[2]))
    g = c.to_graph(zh=True)
    zx.hsimplify.hpivot_simp(g, quiet=True)
    if zx.compare_tensors(g, c.to_graph(zh=True), preserve_scalar=False):
        print("OK", end=';')
    else:
        print("CIRCUIT DIDN'T WORK!!!!")
        print(c.to_qasm())

In [ ]:
def truncate_fraction(f, N=3628800): # N=10!
    if f.denominator < N: return f
    return Fraction(round((f.numerator/f.denominator) * N), N)

In [ ]:
d = os.path.join('..', 'circuits', 'Fast')
print('Circuit'.ljust(30) + '  qubits' + '   gates' + '    Z' + '       H' + '  nf')
for f in os.listdir(d):
    f1 = os.path.join(d,f)
    if f.find('QFTAdd8') != -1: continue # takes too long
    if not os.path.isfile(f1) or f.find('before') == -1: continue
    print(f.ljust(30), end='')
    
    c = zx.Circuit.load(f1)
    for g in c1.gates:
        if isinstance(g, zx.gates.ZPhase):
            g.phase = truncate_fraction(g.phase)
        
    print(str(c.qubits).rjust(8), end='')
    print(str(len(c.gates)).rjust(8), end='')
    g = c.to_graph(zh=True)
    zx.hsimplify.hpivot_simp(g, quiet=True)
    g.normalise()
    z = len([v for v in g.vertices() if g.type(v) == 1])
    h = len([v for v in g.vertices() if g.type(v) == 3])
    print(str(z).rjust(5), end='')
    print(str(h).rjust(8), end='  ')
    
    print(g.qubit_count() * 2 == z)

In [13]:
c0 = zx.Circuit.load('../circuits/Fast/tof_10_before')
c0

Circuit(19 qubits, 51 gates)

In [15]:
g = c0.to_graph(zh=True)
zx.hsimplify.hpivot_simp(g)
g.normalise()
zx.d3.draw(g, auto_hbox=False)

spider_simp: 26. 8. 7.  3 iterations
id_simp: 9.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 1.  1 iterations
id_simp: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 1.  1 iterations
id_simp: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 1.  1 iterations
id_simp: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 1.  1 iterations
id_simp: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 1.  1 iterations
id_simp: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 1.  1 iterations
id_simp: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 1.  1 iterations
id_simp: 1.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 1.  1 iterations
id_simp: 1.  1 iterations


In [ ]:
c1 = zx.Circuit.load('../circuits/Fast/QFTAdd8_before')
for g in c1.gates:
    if isinstance(g, zx.gates.ZPhase):
        g.phase = truncate_fraction(g.phase)
c1

In [ ]:
g = c1.to_graph(zh=True)
zx.hsimplify.hpivot_simp(g, quiet=False)
g.normalise()
print("Normal form: ", end='')
print(g.qubit_count() * 2 == len([v for v in g.vertices() if g.type(v) == 1]))
zx.d3.draw(g)